# Null Value & Missing Data Analysis

## Census Data

- sa2_census.parquet
- sa2_pops.parquet
- sa2_to_postcode.parquet

There are 363 null postcodes in the sa2_to_postcode dataframe out of the 35,040 rows. There are no missing data entries in any of the initial sa2 datasets.

## Consumer Data
- consumer_fraud_probability.parquet
- consumer_tbl.parquet
- consumer_user_details.parquet

There are no null values or missing entries in the consumer dataframes.

## Merchant Data
- merchant_fraud_probability.parquet
- merchant_tbl.parquet

There are no null values or missing entries in the merchant dataframes.

## Transaction Data
- transactions_all.parquet

There are no null values or missing entries in the transaction dataframe.

## Joining Datasets

Joining datasets together, checking for null values after each join and missing data

### Join cons_tbl data with cons_user_det

There are no null values after joining consumer data. 
Also there are no missing data after the first join. That is the number of rows before the join is 499,999 and then there is still 499,999 rows after the join.

### Join cons_join with transaction_all

There are no null values after joining in the transaction data. 
Also there is no missing data after the first join. That is the number of rows in transaction before the join is 14,195,505 and then there is still 14,195,505 rows after the join.

### Join cons_transaction with cons_fp

There is 14,115,157 null values in the consumer fraud probability values out of the 14,195,717 rows.

Also there is an increase in the size of the data after this join. The number of rows before the join is 14,195,505 and then there 14,195,717 rows after the join. So 212 rows are gained in this join, most likely as a result of there being multiple matches in the right table for a single row in the left table.

### Join cons_transaction_with_fraud with merch_tbl

Still the only column with null values is consumer fraud probabilities with 13,543,038 missing out of 13,614,854 rows.

There is a decrease in the size of this data after this join. The number of rows before the join is 14,195,505 whereas after the join there is 13,614,854 rows. This means that there are 580,651 missing rows of data after this join.

### Join cons_transaction_fraud_merchant with merch_fp

After joining the merchant fraud probability, there are 2 columns that have null values. These are consumer_fraud_probability_% and merchant_fraud_probability_%. consumer_fraud_probability_% has 13,543,038 null values and merchant_fraud_probability_% has 13,610,826 null values.

There is also no change in the size of this data after this join. The number of rows before the join is 13,614,854 and remains the same after the join.

## Joining SA2/Census Data

### Join the sa2_to_post to sa2_pops

There are no null values from this join. Also no rows are missing, with 35,040 rows still remaining after the join

### Join the sa2_postcode_and_pops with sa2_census

There are still the null values present from the other previous search (363 postcode values are null). 

Also after the joining, there are still 35,040 rows in the dataframe. No rows have been lost joining the data together.

## Now to join all tables together

No new columns with null values. Still consumer_fraud_probability_% and merchant_fraud_probability_%. consumer_fraud_probability_% has 11,312,983 nulls and merchant_fraud_probability_% has 11,369,563 null values.

There is a decrease in the size of this data after this join. The number of rows before the join is 13,614,854 whereas after the join there is 11,372,905 rows. This means that there are 2,241,949 missing rows of data after this join.

# Outlier Analysis

# Data Insights